In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time 
    #json library
import json
    #Analyst tools
import sys

sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from procurement_lib import GoogleSheet

In [12]:
df_gtin = GoogleSheet("1TBAwmxAoqg6FX79d7wLu3DVtIiOb6gRfkN5fMA_tLNc")
df_gtin = df_gtin.get_as_dataframe('Hoja 1')

In [3]:
query = """
SELECT DISTINCT
    CASE 
    WHEN state = 'Grande SP' THEN 'SPO'
    WHEN state = 'SP Interior' THEN 'VCP'
    WHEN state = 'MG' THEN 'BHZ'
    WHEN state = 'PR' THEN 'CWB'
    ELSE 'NA' END AS site_code,
    i.price_date AS quotation_date,
    CAST(REPLACE(i.price_date, '-', '') AS INTEGER) AS dim_quotation_date,
    i.store_type || ' - ' || i.store AS competitor_name,
    gtin,
    AVG(COALESCE(
        REPLACE(NULLIF(i.regular_price::VARCHAR, ''), ' ', '0')::FLOAT,
        REPLACE(NULLIF(i.price_paid::VARCHAR, ''), ' ', '0')::FLOAT,
        REPLACE(NULLIF(i.price_paid_per_standard_unit_of_measure::VARCHAR, ''), ' ', '0')::FLOAT,
        0.0
    )) AS price
    
FROM lnd_ops.infoprice_product_scraper i
WHERE store_type IN ('Atacado')
 AND gtin IN (7896045503414,7891149011001,7898903113025,7896045506248,7894900010015,78936683,7898903113063,7894900027013,78908901,7894900011609,7894900700015,7896005310090,7891991295017,7896045505340,7896478800043,7891991000826,7891991014908,7891991001342,7891991000727,7892840812423,7891149201006,7896005310120,7896005310113,7896478800012,7896005302088,7894900660333,7897395020217,7896005310168,78905498,7896045506040,7894900030013,611269991000,7896045506873,78933873,7894900681017,7891991000840,7894900701517,7894900010398,7894900660401,7894900660319,7891991011723,7894900531008,7891149108718,7892840800079,7894900700398,7896005310649,7894900530001,7891991303309,7896478800074,7896005302255,7896478800081,7894900660265,7891991014762,7896547501154,7894900093001,7891050000903,7896005310656,7894900030396,7896547501277,7894900681178,7894900050394,7896547501161,7894900093025,7893218000473,7894900094008,7898312042121,7898312044323,7898081291256,7898312041278,7896275900601,7894904014569,7809300052604,7898312042657,7898312045368,7899566300425,7894904009749,7896581301734,7894904015108,7896581300317,7894904578252,7894904013180,7896275900618,7896581322203,7894904572465,7894904255436,7896275940829,7898624690478,7898312040851,7893000444232,7894904018246,7894904005550,7898312041254,7894904018161,7899566301583,7894904578153,7894904197699,7898312043722,7898312041247,7899566302429,7894904006304,7899566300043,7893000018768,7898312042640,7899566304485,7894904003495,7896275940218,7896275900977,7898312040493,7896581352057,7896275920753,7894904009282,7896275940201,7898537400683,7897147211054,7896275900595,7894904208913,7893000024014,7898537401147,7894904230648,7898177100868,7898312041537,7896275921095,7894904015221,7894904691005,7894904034543,7891515532352,7896275949921,7898921567411,7896105800361,7797906054499,7896105800811,7898921567497,7896105800217,7897387100118,7897387101139,7898707750013,7898312045023,7898707750006,7897387105434,7897387101153,7898921567329,7897387102815,7898921567404,7896581353399,7898921567848,7898312042930,7897387105427,7898967047229,7896581307255,7897387101238,7896104992937,7896104999905,7896914011217,7896061915208,7896061915444,7896104995129,7896104992371,7896061920202,7897686300332,7896104991961,7896104993637,7896061920318,7896104992364,7896104996393,7891172421174,7896061934285,7896301802923,1789117243389,7894474200843,7898930672516,7896104994139,7898608574494,7896122305207,7898659280620,7898184631652,7896122300059,7896427701391,7896051111016,7898215151883,7898215152002,7896434921027,7898039680774,7898264360199,7896982102435,7898215151784,7898215152033,7894904008018,7898215152354,7898039680613,7898039680767,7898184631713,7891515410315,7891515593865,7896434920723,7893000079298,7896434921232,7893000394209,7898215152347,7896747800668,7891030300306,7896747800644,7896275950316,7894904268726,7896434921133,7891515550097,7891030300177,7898184630198,7894904929658,7891030300207,7896747800132,7896353300026,7891515979355,7898215151807,7896122301148,7894904271498,7898039680835,7891030300290,7891030300139,7891030300276,7896122301308,7891030002033,7898184630327,7896122301322,7896122300134,7891022638004,7891022242003,7896013102045,7896527700751,7891022100372,7896098900208,7891022101003,7897534800212,7896098900253,7896098905906,7896527700782,7897534801851,7897534801288,7896527700485,7897534800236,7896527702670,7896013142065,7896098909805,7896527700331,7897534848252,7896098900239,7891022640007,7896013102311,7897534854161,7897750783542,7896527700805,7897534826328,7897534821514,7896527702007,7897534800465,7896527701581,7896527700775,7896098900222,7897534826588,7897534822108,7896527700614,7897534805309,7897534800434,7897534800595,7891022860535,7897534832695,7897534801240,7896527702663,7897534801752,7891022860580,7896527700218,7897534800458,7891022861112,7891022860832,7896527700355,7896527701826,7897534859555,7896098904787,7896527702427,7896098904770,7897534805828,7897534800960,7897534801929,7891022471007,7896527700911,7896098902547,7897534800410,7897534819399,7896090100101,7896036090855,7898247780006,7896935810011,7896006751137,7896444118158,7896036097991,7898949924128,7896444198358,7896444118219,7898949924180,7898949924074,7898907037211,7896102501797,7896444198310,7896032501010,7896444110916,7896021300044,7896036097885,7896029310014,8410179099972,7891032011156,7896935810028,7896062400437,7896021314720,7891032013105,7892020000992,7892020200217,7892020220505,7896005800546,7896102502166,7896935830033,7896036092927,7896444111159,7896057800228,7896005800010,7896025803121,7896444110053,7896005801529,7896036096680,7896444199157,7896248100472,7896036094242,7896444199256,7896005801512,7898907037037,7896005800027,7896021314256,7896444196255,7896102501650,7898949924388,7896036097823,7896062400420,7898907037013,7896062400406,7896045102440,7896036000083,7896025802674,7898907037204,7891910000197,7896057800556,7896444110114,7892020200194,7896062400253,7898039680729,7898907037044,7896057800174,7896021314744,7898911082177,7898921818070,7898039680712,7896021300433,7896057800525,7896025802681,7896444196453,7896102501872,7891032014324,7896045102396,7891000248775,7896036098264,7891008367027,7896036097830,7896021300419,7891000460214,7896444196354,7896025802995,7891008123999,7896102509496,7896025802872,7896005803721,7898907037020,7892020000886,7896062400468,7891031412602,7891031409428,7898911082894,7891000107843,7896021302994,7896036096437,7896062400369,7896025802889,7896036097861,7892020200644,7898911082160,7895800400142,7896057800617,7891032012207,7896021300655,7898911082061,7896009301049,7896062400499,7891000253427,7896036097892,7896025803206,7896036090909,7896005800362,7896048210036,7898911082887,7896021314232,7895800400159,7896036094396,7896036097458,7896490288867,7896048284822,7896036098851,7895800400180,7891031409756,7896102501896,7896229600243,7898039680705,7896048200051,7896062400475,7891032011323,7891000336380,7891008209020,7898911082078,7891965132195,7898911082290,7891032011330,7891032014690,7891000003299,7896036096956,7891965129812,7898921818841,7896102504361,7891031405345,7896005803714,7896490288775,7891032015918,7896036096673,7896036099452,8410179100050,7891031116944,7896029300923,7896036093467,7896019602006,7896102502558,7891030300191,7896036090879,7896057800198,7896021303809,7896102500493,7898911082719,7896036000816,7896025802919,7896036097878,7896036097601,7896025803008,7896025803015,7896057800433,7896102502428,7896229600175,7891031116920,7896036000960,7896292343702,7896102502145,7891032013952,7896102502203,7896025801172,7896029312018,7896036096628,7896102503715,7896036096604,7898911082917,7896036096925,7896036094709,7896048284778,7896048255013,7891965120253,7896057800235,7891965159888,7898911082788,7891032012504,7891965120154,7898921818254,7891965220427,7891965120345,7896048284037,7896102502008,7898921818049,7891032015994,7891000248836,7891965220083,7891032015215,7896048258038,7891965122219,7896048258014,7891031405604,7891965129799,7896229600168,7891965121397,7891095154074,7896057800181,7896102503708,7891149010509,7896045506934,7896349600017,7896349600185,7896045512317,7897395020101,7891149200504,7891991010825,7894900660289,7891991294942,7896002100014,9002490100490,7891991015493,7896050200124,7896419720386,7896419716280,7891515220983,7896275910112,7891527000795,7896275920906,7896419720447,7896419719656,7896419715566,7898625902471,7896275920418,7896419720263,7898625900804,7896275940744,7891527962376,7891527044942,7898909698007,7894904015849,7896419717560,7896275981273,7891527069334,7894904014293,7896419715047,7891527000535,7891527962253,7896275590097,7894904577859,7896419714064,7896419719649,7894904018406,7894904577941,7894904575350,7891515474737,7894904578245,7898921567121,7896419724582,7898181470117,7898181470520,7898181470018,7898181470032,7898181470285,7790199604068,7891527064728,7894904955800,7898181471425,7898181470094,7891527064704,7898960283938,7898181470261,7896419720003,7908349900070,7908349900094,7896419720270,7908349900124,7908349900179,7898181470537,7898181470070,7908349900056,7908349900117,7898181475051,7908349900285,7898505140207,7897686300509,7891172433382,7896061931383,7891172151125,7896569405515,7898215151708,7896256601848,7898922071023,7896569405508,7898019060305,7896569400848,7898215152026,7898915414134,7898915415544,7896030518027,7896030521362,7896353301207,7896504306808,7896504305061,7896504307232,7896540904464,7896540900831,7896098904664,7896540900633,7896098904688,7896540902408,7896090300259,7896540900305,7896540902156,7896527702618,7896098905258,7898910095185,7896540902255,7891022080049,7896540904457,7896540901135,7896540900558,7896527700768,7896540900671,7896098903520,7898910095208,7891022860740,7896540900626,7898360980055,7898360980314,7896540900312,7896098904336,7896527700335,7896090300044,7896090300167,7896090301157,7896540900381,7898917671153,7896540900541,7898910095062,7898910095376,7898910095055,7896098900802,7898910095048,7898910095369,7898910095024,7896036090244,7896267000135,7898247780297,7896508200041,7898171400049,7896508200034,7898171400209,7896267004102,7896419431053,7898171400025,7896419436027,7896050600276,7897798800218,7896412819247,7896508200010,7896022205720,7897798800232,7896022207373,7896021800049,7896267000128,7896015910006,7896027920017,7896015910013,7896015912505,7897798800409,7896267004201,7896419431015,7898902299058,7396232115673,7897798800249,7896508200027,7896022200756,7897798800324,7896045102501,7896273900306,7896027910018,7896022205737,7891021006125,7898902299805,7896412800092,7891021005043,7896022205744,7896045102495,7898930142722,7896054901799,7896412800122,7896232115680,7896022205782,7896027910025,7896412800108,7896102500608,7896036097618,7896015976804,7896054901812,7896015973308,7896054907012,7896054909368,7891021006934,7896054907101,7896054901805,7896054909252,7896015976095,8410179100036,7896021810055,7896022200534,7896015910600,7896412803048,7896273900122,7896015976002,7896232115697,7895800400166,7896054907104,7896005806012,7891021006071,7897798800201,7896273900047,7896022200572,7896054900044,7896273900948,7896015976217,7622210573377,7896054908590,7897798800287,7896021800209,7896273908289,7896015922009,7896054909214,7896232115703,7896273900559,7896273900429,7622210812636,7896054905384,7896054909276,7896273900023,7896273900320,7891021005067,7896102583182,7896021810017,7896021822157,7896054907036,7896015910037,7896054904745,7896036096932,7898902299867,7896054906992,7898902299027,7622210573346,7896015930608,7896054907005,7622210858245,7896036099735,7622210573315,7896054909375,7622210570079,7896025803145,7896054900020,7896273900337,7896054906985,7898930141572,7622210566393,7896412800863,7896036097625,7896054906732,7896054906756,7896054906718,7896102500448,7898167881128,7898930142715,7898167881197,7896054908910,7896054907111,7896041173758,7896036094686,7896054906978,7891021006927,7898930142470,7895800116258,7896057800143,7896048200105,7895800116500,7898386200083,7898950286178,7896052604975,7891149103102,7896010006360,7898386200014,7897395040307,7898950286260,7896326100219,7896607100044,7896052603497,7892840812850,7898950286185,78905351,7891991002646,7896685200063,7898915949209,7896741609960,7896005310182,7891991015462,7896005310137,7896547500089,7891991297424,7896000555359,7891991295277,7898367983790,7894900593709,7896005310106,7896000594099,7896273100263,7893218003603,7894900583700,7898367980010,7896000594068,7898915949964,7898651640149,7896000530066,7898651640064,7896000597670,7896000594044,7898651640026,7898651640071,7896000530363,7896000554369,7898651640088,7898651640361,7896000594051,7894900563702,7891991010481,7898651640330,7896000530325,7898651640279,7896000595775,7898651640293,7896000597847,7898651640712,7896005302040,7891991303347,7896000593863,7896000597878,7896000593900,7898651640132,7896000556066,7896000593702,7896000557056,7896000595737,7898651640040,7896607100174,7891991295086,7896000594129,7891991306324,7893218002576,7896000598356,7896514702010,7896000593719,7891991295055,7896000597250,7896000597854,7896000597274,7896000593924,7896000596598,7896000597267,7896000597281,7896000596406,7896000530264,7896000595744,7896514702034,7891149108282,7891991298674,7892840813505,7896000596390,7898063767038,7896000559265,7898651640033,7898063767045,7898208480754,7898345781462,7893000526044,7898466107998,7898537400621,7898959250415,8936044094731,7896419720157,7898345783619,7898302290372,7893000018761,7894904230624,7898537402472,7894904007059,7898915415445,7898537400607,7898641040379,7897232236061,7898542029923,7898302284432,7898537401819,7898966869174,7908344802300,7899567248924,7908344801938,7894904706983,7897232228141,7893000521322,7898902299829,7898641041567,7898466108018,7899567292569,7899566302412,7908344801928,7898966869143,7899567292552,7898934113575,7898966869013,7898966869082,7898966869235,7898303210140,7899567248894,7897232229728,7898537402496,7908344802635,7894904578160,7908344804028,7898537401796,7898641049433,7894904247806,7898537403158,7898537403141,7798304851703,7897387103638,7798304851215,7897387105441,7896532100010,7897387103560,7896419720010,7897387103645,7898921567060,7898921567305,7897387103621,7897387104505,7897387103553,7791688888204,7897387110357,7897518400810,7897248600115,7898950515018,7897518409387,7898921567190,7896594903017,7896594972679,7897387102402,7897518409004,7898922405583,7897518400742,7897518409028,7897518409042,7898044351713,7897518400735,7898921567220,7897518400766,7897518400797,7898922405774,7897518400872,7896105800026,7897518400827,7898922405580,7898044356015,7897518409011,7897518400841,7898044351744,7897518403538,7898044355070,7898922405767,7791688000064,7898325581242,7898505142652,7898918339151,7891172433801,7898505142621,7891172151323,7897686300202,7702458264166,7896061973055,7896066767284,7898080640611,7898925834014,7896569405003,7898603780015,7898024450115,7898288160256,7898902971015,7896030511035,7898934008550,7898934008215,7896275960087,7896030520808,7898288160171,7896030502019,7898041930010,7896353300521,7896030519468,7896353301542,7898039680255,7896434920778,7894904270002,7898934008222,7896275960520,7898039680323,7898902971022,7898039680149,7898039680118,7894904268238,7896051130154,7898215157786,7898039680903,7896051114024,7896005809518,7898215157861,7898955705049,7896051115229,7894904264643,7898692300316,7896051145219,7896051164128,7896051128137,7896098905203,7896524726457,7892141955737,7896524726150,7898952156097,7896524726518,7896527702656,7898952156202,7896527701611,7896524711200,7896524727508,7896527701642,7896098903698,7898952156035,7896098903506,7892141955874,7896098903308,7896527702601,7896098909744,7896527700379,7896527702489,7896524711255,7898952156257,7891022860733,7896527700904,7898952156103,7896527702595,7891022851021,7896527700577,7896527700560,7896098904800,7896527700584,7896098904671,7896098903629,7896524740507,7896098909935,7896084700515,7896070800090,7896116900029,7896276060021,7898247780075,7896894900013,7896089012019,7896116900647,7896006744115,7896070800014,7896006751151,7896005286579,7898276600108,7896006751113,7896116900661,7896070800137,7896248101530,7896006703112,7896089011357,7896070800113,7891080000157,7896110192918,7896110100043,7896084700492,7892020001388,7892020220628,7898931140550,7891018001393,7891080000119,7891018427643,7896248100036,7896053600013,7896248100038,7891018001386,7896006795100,7892020200156,7896205788040,7892020220529,7896025805170,7896116900623,7896053600044,7891018427582,7898276600122,7896006793687,7891032014621,7898416521157,7891910020147,7896005286593,7896005202074,7898167887984,7896490288805,7896084700522,7896804602518,7896894900037,7891018427599,7891018427650,7896006762027,7898167881029,7896894900068,7896412800856,7896412852268,7896006750734,7896006751250,7896005232705,7898167887663,7896005281710,7896110100814,7896070800793,7896005286524,7898416529849,7896070800199,7892020001180,7896430500011,7896005800140,7896894900082,7891910030347,7896116900784,7898950284372,7891910030354,7892020200231,7891959014612,7898931140529,7898416527548,7892020001456,7896116900685,7896894900204,7898416521164,7892020001395,7896116900937,7896084701659,7898903594527,7891032011248,7891032011057,7898167881074,7896894900075,7622210566362,7898167883153,7896248100014,7896005800386,7896205788156,7896009301131,7896224836098,7896116902078,7898167889605,7898167887991,7896102502173,7898416529856,7896005277317,7898416521270,7896025803091,7896005281697,7896105900153,7896116900197,7898416521232,7896894900198,7896248101073,7898167889032,7892020200620,7896005222638,7896057801010,7891080013423,7896005275399,7895800400227,7896102583205,7898937864948,7896005286555,7891910020192,7896070800106,7896005275832,7893500036012,7896025802858,7896025800052,7896005212714,7898937864115,7896005808733,7896105900160,7896009301209,7896205788217,7896005212721,7898416529610,7891000313022,7896005806180,7896005275382,7898416527623,7896025805484,7896292321431,7898167887731,7891008198027,7896524705551,7622210878946,7891000321669,7891080106019,7896105900221,7896025803688,7893500018452,7896005286609,7896430502312,7891000092613,7896045111398,7896105900351,7891080007682,7891000251539,7898167887557,7891032014287,7898416521072,7895800400173,7896036098813,7898416521096,7896084700508,7898937864955,7891008114126,7891000312926,7898937864788,7896025802865,7896102500530,7896070800151,7896025804074,7622210857255,7896292302693,7896894900235,7898167883016,7896434921201,7896045103003,7891008273502,7896205788095,7898167887502,7790070225580,7896025801196,7896005212615,7898950284396,7896005800157,7896412852275,7896009301216,7898416520082,7898416520761,7891000538500,7896110195216,7891000464915,7896110195162,7896005286531,7896006711155,7898416522093,7891000312971,7896430500059,7896025803718,7896005800829,7896025803695,7898950284389,7898416521089,7898416524547,7896070802193,7891080013720,7896025800250,7898416521119,7896036097595,7898902299584,7896292300460,7898416520808,7898416524639,7896005801819,7898416521300,7898937864078,7896036099353,7898416520778,7896205788033,7896025805040,7898416529764,7898946867077,7896036099346,7896025800069,7896025805071,7896025803701,7898416524851,7898416520051,7896025805361,7891965120642,7898416529917,7898946867107,7896036099223,7891965129898,7896036096949,7896036099230,7891965120543,7896036000144,7891965410217,7896036096994,7891032011224,7891080151330,7898416522086,7896036000151,7896025804883,7898416520181,7896070800052,7898946867084,7891965431021,7896186300118,7896186302884,7896186302945,7896186300507,7896547501178,7898651640354,7898952156042,7896419701637,7898537430068,7896419714224,7896419722663,7893000973817,7893000106420,7894904769889,7896419722656,7896419713852,7908344802512,7894904273294,7897232229704,7893000387096,7897232221838,7893000913912,7897232235019,7797906000526,8711571072519,7897387106080,7897387101221,7897387101436,7897387101122,7898922405750,7897518409110,7896089407655,7898505142645,7898608572882,7894474003062,7896982102213,7896030511028,7896030501012,7898378440022,2145300016004,7896030518157,7896353301535,1790834480281,7898039680325,7894904271481,7892141956093,7896524700389,7896524705858,7896524705063,7896524740408,7896090300235,7896527700386,7896524700372,7898952156189,7896235100027,7896235100218,7896070802209,7896102000313,7896036099216,7896292321434,7896036099339,7898416520785,7896005279489,7896292321465,7896084701635,7898167887854,7896292324473,7898416522055,7896292330467)
GROUP BY 1,2,3,4,5
""".format(skus=tuple(df_gtin.GTIN.unique().tolist()))

df = read_connection_data_warehouse.run_read_prod_query(query)

df['quotation_date'] = pd.to_datetime(df['quotation_date'])
df[['gtin']] = df[['gtin']].astype('float')

In [4]:
df.site_code.value_counts()

SPO    339101
VCP    191896
BHZ     65645
CWB     54048
Name: site_code, dtype: int64

In [5]:
df2 = pd.merge(df, df_gtin, left_on=['gtin','site_code'], right_on=['GTIN','city'], how='left')
df2 = df2.drop(['city', 'GTIN'], axis=1)
df2 = df2.dropna().reset_index(drop=True)

df2['lifetime'] = 8

In [6]:
# Ensure dataframe is sorted by 'quotation_date'
df2 = df2.sort_values(by='quotation_date')

# Generate the required rows for missing dates
new_rows = []

for (competitor, source_id), group in df2.groupby(['competitor_name', 'source_id']):
    group = group.sort_values(by='quotation_date')
    last_known_price = None
    last_known_date = None
    lifetime = 8
    
    for current_index in range(len(group)):
        current_date = group.iloc[current_index]['quotation_date']
        price = group.iloc[current_index]['price']
        
        # If this is not the first iteration, fill in missing dates
        if last_known_date is not None:
            days_diff = (current_date - last_known_date).days
            if days_diff > 1:
                for j in range(1, min(days_diff, lifetime + 1)):
                    new_date = last_known_date + timedelta(days=j)
                    new_row = {
                        'site_code': group.iloc[current_index]['site_code'],
                        'quotation_date': new_date,
                        'dim_quotation_date': int(new_date.strftime('%Y%m%d')),
                        'competitor_name': competitor,
                        'source_id': source_id,
                        'product_name': group.iloc[current_index]['product_name'],
                        'price': last_known_price,
                        'lifetime': lifetime - j
                    }
                    new_rows.append(new_row)
                    
                    # Stop if we reach a new datapoint date
                    if new_date + timedelta(days=1) == current_date:
                        break
        
        # Update the last known values and reset lifetime
        last_known_price = price
        last_known_date = current_date
        lifetime = 8  # Reset lifetime

    # After processing all known dates for the group, continue generating rows until lifetime reaches 0
    while lifetime > 0:
        last_known_date += timedelta(days=1)
        new_row = {
            'site_code': group.iloc[-1]['site_code'],
            'quotation_date': last_known_date,
            'dim_quotation_date': int(last_known_date.strftime('%Y%m%d')),
            'competitor_name': competitor,
            'source_id': source_id,
            'product_name': group.iloc[-1]['product_name'],
            'price': last_known_price,
            'lifetime': lifetime - 1
        }
        new_rows.append(new_row)
        lifetime -= 1

# Append new rows to the dataframe
df2 = df2.append(new_rows, ignore_index=True)

# Sort the final dataframe
df2 = df2.sort_values(by=['competitor_name', 'source_id', 'quotation_date'])
df2['replica'] = df2['lifetime'] == 8

In [7]:
df2.to_csv("all_infoprice_info_no_out_by_store.csv", index = False)

In [8]:
df2 = pd.read_csv("all_infoprice_info_no_out_by_store.csv")

# Convert the quotation_date column to datetime format
df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])
df2['product_name'] = df2['site_code'] + " || " + df2['product_name']

In [9]:
import mplcursors
import plotly.express as px
from ipywidgets import interact, widgets
from scipy.stats import median_abs_deviation

# OUTLIER GABI

In [12]:
#df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])

# Ensure DataFrame is sorted by source_id and quotation_date
df2 = df2.sort_values(by=['source_id', 'quotation_date'])

# Function to calculate the MAD
def trimmed_mad(series, trim=0.1):
    # Calculate the trimmed mean
    trimmed_median = np.median(series)
    trimmed_series = series[(series >= np.percentile(series, trim * 100)) & (series <= np.percentile(series, (1 - trim) * 100))]
    trimmed_median = np.median(trimmed_series)
    
    # Calculate the MAD using the trimmed median
    mad = np.median(np.abs(trimmed_series - trimmed_median))
    
    return mad

# Create a rolling window function that uses the last 7 days of data for each product
def rolling_median_mad(group):
    group = group.sort_values(by='quotation_date')
    group.set_index('quotation_date', inplace=True)
    # Calculate rolling median and MAD with a 7-day window
    group['rolling_median'] = group['price'].rolling(window='7D', min_periods=5).median()
    group['rolling_mad'] = group['price'].rolling(window='7D', min_periods=5).apply(trimmed_mad, raw=True)
    
    group.reset_index(inplace=True)
    return group

# Apply the rolling calculations separately for each source_id
df2 = df2.groupby('source_id').apply(rolling_median_mad).reset_index(drop=True)

# Compute robust Z-scores for each entry
df2['robust_z_score'] = (df2['price'] - df2['rolling_median']) / (1.4826 * df2['rolling_mad'])

# Identify outliers with a threshold of 3
threshold = 3
df2['outlier7T'] = np.abs(df2['robust_z_score']) > threshold

In [13]:
#df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])

# Ensure DataFrame is sorted by source_id and quotation_date
df2 = df2.sort_values(by=['source_id', 'quotation_date'])

# Function to calculate the MAD
def mad(series):
    return np.median(np.abs(series - np.median(series)))

# Create a rolling window function that uses the last 7 days of data for each product
def rolling_median_mad(group):
    group = group.sort_values(by='quotation_date')
    group.set_index('quotation_date', inplace=True)
    # Calculate rolling median and MAD with a 7-day window
    group['rolling_median'] = group['price'].rolling(window='7D', min_periods=5).median()
    group['rolling_mad'] = group['price'].rolling(window='7D', min_periods=5).apply(mad, raw=True)
    
    group.reset_index(inplace=True)
    return group

# Apply the rolling calculations separately for each source_id
df2 = df2.groupby('source_id').apply(rolling_median_mad).reset_index(drop=True)

# Compute robust Z-scores for each entry
df2['robust_z_score'] = (df2['price'] - df2['rolling_median']) / (1.4826 * df2['rolling_mad'])

# Identify outliers with a threshold of 3
threshold = 3
df2['outlier7'] = np.abs(df2['robust_z_score']) > threshold

In [14]:
df2.to_csv("all_infoprice_store_outliers.csv", index = False)

## RETOMAR ACA

In [2]:
df2 = pd.read_csv("all_infoprice_store_outliers.csv")

df2['quotation_date'] = pd.to_datetime(df2['quotation_date'])

In [3]:
df2.outlier7T.value_counts()

False    745069
True     103213
Name: outlier7T, dtype: int64

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

def plot_product_price_trend(product_name, outlier):
    df2_filtered = df2[df2['product_name'] == product_name]
    
    # Identify outliers based on robust_z_score
    outliers = df2_filtered[df2_filtered[f'outlier{outlier}'] == True]

    # Create the figure
    fig = go.Figure(layout=go.Layout(height=1500))

    # Add line plots for each competitor
    for competitor in df2_filtered['competitor_name'].unique():
        competitor_data = df2_filtered[df2_filtered['competitor_name'] == competitor]
        competitor_data_backup = competitor_data.copy()
        try:
            competitor_data.set_index('quotation_date',inplace=True)
            competitor_data = competitor_data.asfreq('D') 
            competitor_data.reset_index(inplace = True)
        except:
            #print(competitor_data.index)    
            print(competitor)
            competitor_data = competitor_data_backup.copy()
            
        fig.add_trace(go.Scatter(x=competitor_data['quotation_date'], y=competitor_data['price'], 
                                 mode='lines', name=competitor, connectgaps=False))

    # Add scatter plot for outliers
    fig.add_trace(go.Scatter(x=outliers['quotation_date'], y=outliers['price'], mode='markers',
                             marker=dict(color='red', size=10), name='Outliers'))

    fig.update_yaxes(title_text='Price', tickprefix='$', tickformat='.2f', title_font=dict(size=18))
    fig.update_layout(title=f'Price Trend for Product Name: {product_name}',
                      xaxis_title='Date', yaxis_title='Price',
                      legend_title='Competitor', legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5))  # Adjust height here

    fig.show()
# Get unique product names
product_names = pd.Series(df2.product_name.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=product_names, description='Product Name:')
dropdown2 = widgets.Dropdown(options=['3','3T','7','7T'], description='Outlier:')

# Interact function to update plot based on dropdown selection
interact(plot_product_price_trend, product_name=dropdown,outlier=dropdown2);

interactive(children=(Dropdown(description='Product Name:', options=('BHZ || Achocolatado em Pó Nescau 2.0 Sac…

In [4]:
df2.loc[df2.source_id == 384239].outlier7T.value_counts()

False    1603
True      137
Name: outlier7T, dtype: int64

# CONTINUACION 1

In [4]:
summary_df = df2.groupby(['quotation_date', 'source_id','product_name'])['price'].describe(percentiles=[0.10, 0.25, 0.5, 0.75]).reset_index()
summary_df = summary_df[['quotation_date', 'source_id','product_name', 'min', '10%', '25%', '50%', '75%', 'max']]
summary_df.columns = ['quotation_date', 'source_id','product_name', 'min', 'p10','q1', 'median', 'q3', 'max'] #ESTABA CON 3T

In [5]:
summary_df

,quotation_date,source_id,product_name,min,p10,q1,median,q3,max
0,2024-02-10,72318.0,SPO || Feijão Preto Camil 1Kg,8.1900,8.283000,8.590000,8.985000,9.679000,13.300
1,2024-02-10,72329.0,SPO || Feijão Preto Kicaldo 1Kg,8.2925,8.470250,8.602750,8.832250,9.140625,10.840
2,2024-02-10,72488.0,SPO || Feijão Carioca Camil 1Kg,6.9900,7.590000,7.841964,8.095278,8.802500,12.100
3,2024-02-10,72497.0,SPO || Feijão Carioca Kicaldo 1Kg,6.7900,7.577813,7.903259,8.026750,8.377500,9.790
4,2024-02-10,73456.0,SPO || Açúcar Refinado União 1Kg,3.8900,4.016643,4.291786,4.469000,4.890000,5.750
...,...,...,...,...,...,...,...,...,...
154585,2024-05-30,652553.0,CWB || Lentilha 400g - Caldo Bom,13.1900,13.190000,13.190000,13.190000,13.190000,13.190
154586,2024-05-30,652555.0,CWB || Farinha de Milho Amarela 1Kg - Caldo Bom,4.9800,4.980000,4.980000,4.980000,4.980000,4.980
154587,2024-05-30,652559.0,CWB || Farofa Mix Crocante 250g - Caldo Bom,3.8900,3.890000,3.890000,3.890000,3.890000,3.890
154588,2024-05-30,652561.0,CWB || Fubá Amarelo 1Kg - Caldo Bom,5.7900,5.790000,5.790000,5.790000,5.790000,5.790


In [6]:
query = """
SELECT
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS fecha_pricing,
    sup.source_id,
    SUM(fs.product_quantity_x_step_unit) AS cant,
    SUM(fs.product_price*fs.product_quantity_x_step_unit)/cant AS precio,
    SUM(fs.product_price_discount*fs.product_quantity_x_step_unit)/cant AS precio_neto,
    SUM(dtd_cost_local*fs.product_quantity_x_step_unit)/cant AS costo,
    SUM(min_price*fs.product_quantity_x_step_unit)/cant AS bench,
    (1-(costo/precio))*100.00 AS margin,
    (1-(costo/precio_neto))*100.00 AS net_margin


FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
LEFT JOIN dpr_cross_business.int_dtd_cost m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date_dtd = fs.dim_submitted_date
LEFT JOIN dpr_product_pricing.obt_benchmark_product_prices  pb  ON pb.stock_unit_id = sup.stock_unit_id AND DATE(fs.order_submitted_date) = pb.benchmark_date

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    AND cat.super_category <> 'Fruver'
    AND s.identifier_value IN ('SPO','CWB','VCP','BHZ')
    AND DATE(fs.order_submitted_date) >= '2024-02-10'
    AND DATE(fs.order_submitted_date) < '2024-06-01'

GROUP BY 1,2,3--,4,5,6,7,8,9
--HAVING gpi < 150 AND gpi > 50
"""
df_zk = run_read_dwd_query(query)
df_zk['fecha_pricing'] = pd.to_datetime(df_zk['fecha_pricing'])

In [7]:
# Merge the DataFrames
merged_df = pd.merge(summary_df, df_zk, left_on=['quotation_date', 'source_id'], right_on=['fecha_pricing', 'source_id'], how='left')

# Drop redundant columns
merged_df.drop(columns=['fecha_pricing'], inplace=True)

In [9]:
# merged_df.to_csv("INFO_grafica.csv",index=False)

In [2]:
merged_df = pd.read_csv("INFO_grafica.csv")

merged_df['quotation_date'] = pd.to_datetime(merged_df['quotation_date'])

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact

# Function to plot data for a single product with additional metrics
def plot_product_prices_with_complement(product_name):
    product_data = merged_df[merged_df['product_name'] == product_name]

    fig = go.Figure()

    # Plot the summary statistics
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['min'], mode='lines', name='Min'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q1'], mode='lines', name='Q1'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['p10'], mode='lines', name='P10'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['median'], mode='lines', name='Median'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['q3'], mode='lines', name='Q3'))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['max'], mode='lines', name='Max'))
    
    # Plot the additional values with connectgaps=True
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['precio'], mode='markers+lines', name='Sale Price', line=dict(dash='dash'), marker=dict(symbol='circle'), connectgaps=True))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['precio_neto'], mode='markers+lines', name='NET Sale Price', line=dict(dash='dash'), marker=dict(symbol='circle'), connectgaps=True))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['bench'], mode='markers+lines', name='Benchmark', line=dict(dash='dash'), marker=dict(symbol='square'), connectgaps=True))
    fig.add_trace(go.Scatter(x=product_data['quotation_date'], y=product_data['costo'], mode='markers+lines', name='Cost', line=dict(dash='dash'), marker=dict(symbol='triangle-up'), connectgaps=True))

    fig.update_layout(
        title=f'Price Summary and Additional Metrics for Product ID: {product_name}',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Metrics',
        height=800,
        template='plotly_white'
    )

    fig.show()

# Get unique product IDs
product_names = pd.Series(merged_df.product_name.unique()).sort_values()

# Create dropdown widget
dropdown = widgets.Dropdown(options=product_names, description='Product Name:')

# Interact function to update plot based on dropdown selection
interact(plot_product_prices_with_complement, product_name=dropdown)

interactive(children=(Dropdown(description='Product Name:', options=('BHZ || Achocolatado em Pó Nescau 2.0 Sac…

<function __main__.plot_product_prices_with_complement(product_name)>

# CONTINUACION

In [22]:
df3 = df2.groupby(by=['site_code','source_id', 'product_name']).agg(
    data_points=('site_code', 'count'),
    days=('dim_quotation_date', pd.Series.nunique),
    competitors=('competitor_name', pd.Series.nunique),
    price_avg=('price', np.mean),
    price_median=('price', np.median),
    price_q1=('price', lambda x: x.quantile(0.25)),
    price_q3=('price', lambda x: x.quantile(0.75)),
    replica_points=('replica', np.sum)
).reset_index()

In [24]:
query = """
SELECT
    --DATE(fs.order_submitted_date) AS fecha,
    sup.source_id,
    --sup.description AS product_name,
    --AVG(cogs_p_mtd) AS costo,
    SUM(fs.gmv_pxq_local)/SUM(fs.product_quantity_x_step_unit) AS price,
    SUM(fs.product_price_discount*fs.product_quantity_x_step_unit)/SUM(fs.product_quantity_x_step_unit) AS price_net--,
    --SUM(fs.gmv_pxq_local)/4.75 AS gmv_usd,
    --100.00*gmv_usd/sum(SUM(fs.gmv_pxq_local)/4.75) OVER (PARTITION BY s.identifier_value) AS gmv_mix,
    --100.00*gmv_usd/ig.gmv_total AS gmv_mix_all
    --SUM(COALESCE(fsd.product_discount,0))/4.75 AS discount_applied,
    --100.00*discount_applied/gmv_usd AS per_dct,
    -- (1-(SUM(cogs_p_mtd*fs.product_quantity_x_step_unit)/SUM(fs.product_price*fs.product_quantity_x_step_unit)))*100.00 AS margin,
    -- gmv_usd*margin/100.00 AS cash_margin,
    -- cash_margin-discount_applied AS net_cash_margin,
    -- 100.00*net_cash_margin/gmv_usd AS net_margin
    
FROM dpr_sales.fact_sales                   fs
--INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
--INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
--INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
--LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
INNER JOIN dpr_shared.dim_stock_unit        su  ON su.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_stock_unit        sup  ON nvl(nullif(su.source_parent_id,0),su.source_id) = sup.source_id
--LEFT JOIN dpr_cross_business.fact_cross_business_insights m ON m.dim_stock_unit = sup.stock_unit_id AND m.dim_date = fs.dim_submitted_date
--LEFT JOIN dpr_product_pricing.obt_benchmark_product_prices  pb  ON pb.stock_unit_id = sup.stock_unit_id AND DATE(fs.order_submitted_date) = pb.benchmark_date
--JOIN info_gmv ig ON s.identifier_value <> 0
WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    --AND dp.is_slot = 'false'
    AND fs.gmv_pxq_local > 0
    --AND cogs_p_mtd > 0
    --AND (sup.source_id in {skus} or su.source_id in {skus})
    AND s.identifier_value IN ('SPO','VCP','CWB','BHZ')
    AND DATE(fs.order_submitted_date) >= '2024-02-10'
    AND DATE(fs.order_submitted_date) <= '2024-05-23'
GROUP BY 1--,2--,s.identifier_value,ig.gmv_total--,,3,4,5,6,7,8,9
""".format(skus = tuple(df2.source_id.unique()))

df_info = run_read_dwd_query(query)

In [26]:
dfi2 = pd.merge(df3, df_info[['source_id', 'price', 'price_net']], left_on=['source_id'], right_on=['source_id'], how='left')

In [29]:
dfi2.head()

,site_code,source_id,product_name,data_points,days,competitors,price_avg,price_median,price_q1,price_q3,replica_points,price,price_net
0,BHZ,251792.0,BHZ || Açucar refinado União 1kg,678,97,13,5.614390,5.190000,4.890000,5.820,229,4.8396,4.8396
1,BHZ,251895.0,BHZ || Leite Condensado Triangulo Mineiro 395g,671,96,14,4.498044,4.290000,3.980000,4.790,273,5.4647,5.4600
2,BHZ,251946.0,BHZ || Óleo de Soja ABC 900ml,1422,97,27,5.636634,5.526667,5.270668,5.990,731,5.3042,5.0686
3,BHZ,252145.0,BHZ || Óleo de Soja Veleiro 900ml,640,97,8,5.197795,5.190000,4.980000,5.390,373,5.3136,5.1308
4,BHZ,257901.0,BHZ || Arroz Branco Camil Food Service 5kg,76,76,1,33.682500,33.590000,33.590000,34.125,33,30.6292,29.4118


In [31]:
dfi2.to_excel("INFO_FINAL.xlsx",index=False)

In [6]:
query = """
select
    cpp.collected_product_prices_id,
    site.identifier_value as region_code,
    quotation_date.full_date,
    su.source_id,
    cpp.product_selected_price::float as price
from dpr_product_pricing.fact_collected_product_prices cpp
    inner join dpr_shared.dim_date quotation_date
        on cpp.dim_quotation_date = quotation_date.date_id
    inner join dpr_shared.dim_time quotation_time
        on cpp.dim_quotation_time = quotation_time.time_id
    inner join dpr_shared.dim_site site
        on cpp.dim_site = site.site_id
    inner join dpr_shared.dim_category cat
        on cpp.dim_category = cat.category_id
    inner join dpr_product_pricing.dim_product_outlier_type outlier_type
        on cpp.dim_outlier_type = outlier_type.outlier_type_id
    inner join dpr_product_pricing.dim_product_source_type source_type
        on cpp.dim_source_type = source_type.source_type_id
    inner join dpr_product_pricing.dim_product_competitor competitor
        on cpp.dim_competitor = competitor.competitor_id
    inner join dpr_product_pricing.dim_product_competitor_type competitor_type
        on(
            case
                when cpp.super_category = 'Fruver'
                    then competitor.product_competitor_type_id_fruver = competitor_type.competitor_type_id
                when cpp.super_category = 'Multicategoría'
                    then competitor.product_competitor_type_id_multicategoria = competitor_type.competitor_type_id
            end
        )
    inner join dpr_shared.dim_stock_unit su
        on cpp.dim_stock_unit = su.stock_unit_id
where quotation_date.full_date >= '2024-03-01'
    AND quotation_date.full_date < '2024-05-01'
    and site.identifier_value IN ('SPO','CWB','VCP','BHZ')
"""
df_zkkkkk = run_read_dwd_query(query)

In [7]:
df_zkkkkk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343710 entries, 0 to 343709
Data columns (total 5 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   collected_product_prices_id  343710 non-null  int64  
 1   region_code                  343710 non-null  object 
 2   full_date                    343710 non-null  object 
 3   source_id                    343710 non-null  int64  
 4   price                        343710 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 13.1+ MB


In [10]:
summary_df2 = df_zkkkkk.groupby(['full_date', 'source_id'])['price'].describe(percentiles=[0.10, 0.25, 0.5, 0.75]).reset_index()
summary_df2 = summary_df2[['full_date', 'source_id', 'min', '10%', '25%', '50%', '75%', 'max']]
summary_df2.columns = ['full_date', 'source_id', 'min', 'p10','q1', 'median', 'q3', 'max'] #ESTABA CON 3T

In [11]:
summary_df2.to_excel("INFO_cuartiles_zk.xlsx",index=False)